In [ ]:
from lxml import etree
from lxml.etree import QName
from collections import defaultdict
import re
from IPython.display import display, HTML

# 命名空間定義
NSMAP = {
    "cbc": "urn:oecd:ties:cbc:v2",
    "iso": "urn:oecd:ties:isocbctypes:v1",
    "xsi": "http://www.w3.org/2001/XMLSchema-instance"
}

# ✅ 定義 structure（加入 Sequence）
structure_sorted = [
    {'Path': '/CBC_OECD', 'Name': 'CBC_OECD', 'Type': '', 'Depth': 0, 'Leaf': 'N', 'Sequence': 1},

    # MessageSpec 區塊
    {'Path': '/CBC_OECD/MessageSpec', 'Name': 'MessageSpec', 'Type': 'TwCbcMessageSpec_Type', 'Depth': 1, 'Leaf': 'N', 'Sequence': 1},
    {'Path': '/CBC_OECD/MessageSpec/SendingEntityIN', 'Name': 'SendingEntityIN', 'Type': 'TwCbcBAN_Type', 'Depth': 2, 'Leaf': 'Y', 'Sequence': 1},
    {'Path': '/CBC_OECD/MessageSpec/currCode', 'Name': 'currCode', 'Type': '{urn:oecd:ties:isocbctypes:v1}currCode_Type', 'Depth': 2, 'Leaf': 'Y', 'Sequence': 2},

    # CbcBody 區塊
    {'Path': '/CBC_OECD/CbcBody', 'Name': 'CbcBody', 'Type': 'TwCbcBody_Type', 'Depth': 1, 'Leaf': 'N', 'Sequence': 2},
 
    # ReportingEntity 內的 Entity 結構
    {'Path': '/CBC_OECD/CbcBody/ReportingEntity', 'Name': 'ReportingEntity', 'Type': 'TwCbcCorrectableReportingEntity_Type', 'Depth': 2, 'Leaf': 'N', 'Sequence': 1},
    {'Path': '/CBC_OECD/CbcBody/ReportingEntity/Entity', 'Name': 'Entity', 'Type': 'OrganisationParty_Type', 'Depth': 3, 'Leaf': 'N', 'Sequence': 1},
    {'Path': '/CBC_OECD/CbcBody/ReportingEntity/Entity/ResCountryCode', 'Name': 'ResCountryCode', 'Type': '{urn:oecd:ties:isocbctypes:v1}CountryCode_Type', 'Depth': 4, 'Leaf': 'Y', 'Sequence': 1},
    {'Path': '/CBC_OECD/CbcBody/ReportingEntity/Entity/TIN', 'Name': 'TIN', 'Type': 'StringMin1Max200_Type', 'Depth': 4, 'Leaf': 'Y', 'Sequence': 2},
    {'Path': '/CBC_OECD/CbcBody/ReportingEntity/Entity/IN', 'Name': 'IN', 'Type': 'StringMin1Max200_Type', 'Depth': 4, 'Leaf': 'Y', 'Sequence': 3},
    {'Path': '/CBC_OECD/CbcBody/ReportingEntity/Entity/Name', 'Name': 'Name', 'Type': 'NameOrganisation_Type', 'Depth': 4, 'Leaf': 'Y', 'Sequence': 4},
    {'Path': '/CBC_OECD/CbcBody/ReportingEntity/Entity/Address', 'Name': 'Address', 'Type': 'Address_Type', 'Depth': 4, 'Leaf': 'N', 'Sequence': 5},
    {'Path': '/CBC_OECD/CbcBody/ReportingEntity/Entity/Address/AddressFree', 'Name': 'AddressFree', 'Type': 'StringMin1Max4000_Type', 'Depth': 5, 'Leaf': 'Y', 'Sequence': 1},

    # ReportingEntity 內其他欄位
    {'Path': '/CBC_OECD/CbcBody/ReportingEntity/NameMNEGroup', 'Name': 'NameMNEGroup', 'Type': 'StringMin1Max200_Type', 'Depth': 3, 'Leaf': 'Y', 'Sequence': 2},
    {'Path': '/CBC_OECD/CbcBody/ReportingEntity/NameMNEGroupCHi', 'Name': 'NameMNEGroupCHi', 'Type': '', 'Depth': 3, 'Leaf': 'Y', 'Sequence': 3},
    {'Path': '/CBC_OECD/CbcBody/ReportingEntity/ReportingRole', 'Name': 'ReportingRole', 'Type': 'CbcReportingRole_EnumType', 'Depth': 3, 'Leaf': 'Y', 'Sequence': 4},

    # ReportingPeriod 區塊
    {'Path': '/CBC_OECD/CbcBody/ReportingEntity/ReportingPeriod', 'Name': 'ReportingPeriod', 'Type': '', 'Depth': 3, 'Leaf': 'N', 'Sequence': 5},
    {'Path': '/CBC_OECD/CbcBody/ReportingEntity/ReportingPeriod/StartDate', 'Name': 'StartDate', 'Type': '{http://www.w3.org/2001/XMLSchema}date', 'Depth': 4, 'Leaf': 'Y', 'Sequence': 1},
    {'Path': '/CBC_OECD/CbcBody/ReportingEntity/ReportingPeriod/EndDate', 'Name': 'EndDate', 'Type': '{http://www.w3.org/2001/XMLSchema}date', 'Depth': 4, 'Leaf': 'Y', 'Sequence': 2},

    # CbcReports 區塊
    {'Path': '/CBC_OECD/CbcBody/CbcReports', 'Name': 'CbcReports', 'Type': 'TwCbcCorrectableCbcReport_Type', 'Depth': 2, 'Leaf': 'N', 'Sequence': 2},
    {'Path': '/CBC_OECD/CbcBody/CbcReports/ResCountryCode', 'Name': 'ResCountryCode', 'Type': 'CountryCode_Type', 'Depth': 3, 'Leaf': 'Y', 'Sequence': 1},
    {'Path': '/CBC_OECD/CbcBody/CbcReports/Summary', 'Name': 'Summary', 'Type': '', 'Depth': 3, 'Leaf': 'N', 'Sequence': 2},
    {'Path': '/CBC_OECD/CbcBody/CbcReports/Summary/Revenues', 'Name': 'Revenues', 'Type': '', 'Depth': 4, 'Leaf': 'N', 'Sequence': 3},
    {'Path': '/CBC_OECD/CbcBody/CbcReports/Summary/Revenues/Unrelated', 'Name': 'Unrelated', 'Type': 'TwCbcMonAmnt16_Type', 'Depth': 5, 'Leaf': 'Y', 'Sequence': 4},
    {'Path': '/CBC_OECD/CbcBody/CbcReports/Summary/Revenues/Related', 'Name': 'Related', 'Type': 'TwCbcMonAmnt16_Type', 'Depth': 5, 'Leaf': 'Y', 'Sequence': 5},
    {'Path': '/CBC_OECD/CbcBody/CbcReports/Summary/Revenues/Total', 'Name': 'Total', 'Type': 'TwCbcMonAmnt16_Type', 'Depth': 5, 'Leaf': 'Y', 'Sequence': 6},
    {'Path': '/CBC_OECD/CbcBody/CbcReports/Summary/ProfitOrLoss', 'Name': 'ProfitOrLoss', 'Type': 'TwCbcMonAmnt16_Type', 'Depth': 4, 'Leaf': 'Y', 'Sequence': 7},
    {'Path': '/CBC_OECD/CbcBody/CbcReports/Summary/TaxPaid', 'Name': 'TaxPaid', 'Type': 'TwCbcMonAmnt16_Type', 'Depth': 4, 'Leaf': 'Y', 'Sequence': 8},
    {'Path': '/CBC_OECD/CbcBody/CbcReports/Summary/TaxAccrued', 'Name': 'TaxAccrued', 'Type': 'TwCbcMonAmnt16_Type', 'Depth': 4, 'Leaf': 'Y', 'Sequence': 9},
    {'Path': '/CBC_OECD/CbcBody/CbcReports/Summary/Capital', 'Name': 'Capital', 'Type': 'TwCbcMonAmnt16_Type', 'Depth': 4, 'Leaf': 'Y', 'Sequence': 10},
    {'Path': '/CBC_OECD/CbcBody/CbcReports/Summary/Earnings', 'Name': 'Earnings', 'Type': 'TwCbcMonAmnt16_Type', 'Depth': 4, 'Leaf': 'Y', 'Sequence': 11},
    {'Path': '/CBC_OECD/CbcBody/CbcReports/Summary/NbEmployees', 'Name': 'NbEmployees', 'Type': '', 'Depth': 4, 'Leaf': 'Y', 'Sequence': 12},
    {'Path': '/CBC_OECD/CbcBody/CbcReports/Summary/Assets', 'Name': 'Assets', 'Type': 'TwCbcMonAmnt16_Type', 'Depth': 4, 'Leaf': 'Y', 'Sequence': 13},
    {'Path': '/CBC_OECD/CbcBody/CbcReports/ConstEntities', 'Name': 'ConstEntities', 'Type': '', 'Depth': 3, 'Leaf': 'N', 'Sequence': 14},
    {'Path': '/CBC_OECD/CbcBody/CbcReports/ConstEntities/ConstEntity', 'Name': 'ConstEntity', 'Type': 'TwCbcOrganisationParty_Type', 'Depth': 4, 'Leaf': 'N', 'Sequence': 15},
    {'Path': '/CBC_OECD/CbcBody/CbcReports/ConstEntities/ConstEntity/ResCountryCode', 'Name': 'ResCountryCode', 'Type': 'CountryCode_Type', 'Depth': 5, 'Leaf': 'Y', 'Sequence': 16},
    {'Path': '/CBC_OECD/CbcBody/CbcReports/ConstEntities/ConstEntity/TIN', 'Name': 'TIN', 'Type': 'StringMin1Max200_Type', 'Depth': 5, 'Leaf': 'Y', 'Sequence': 17},
    {'Path': '/CBC_OECD/CbcBody/CbcReports/ConstEntities/ConstEntity/Name', 'Name': 'Name', 'Type': 'NameOrganisation_Type', 'Depth': 5, 'Leaf': 'Y', 'Sequence': 18},
    {'Path': '/CBC_OECD/CbcBody/CbcReports/ConstEntities/ConstEntity/Address', 'Name': 'Address', 'Type': 'Address_Type', 'Depth': 5, 'Leaf': 'N', 'Sequence': 19},
    {'Path': '/CBC_OECD/CbcBody/CbcReports/ConstEntities/ConstEntity/Address/AddressFree', 'Name': 'AddressFree', 'Type': 'StringMin1Max4000_Type', 'Depth': 6, 'Leaf': 'Y', 'Sequence': 20},
    {'Path': '/CBC_OECD/CbcBody/CbcReports/ConstEntities/BizActivities', 'Name': 'BizActivities', 'Type': 'CbcBizActivityType_EnumType', 'Depth': 4, 'Leaf': 'Y', 'Sequence': 21},
    {'Path': '/CBC_OECD/CbcBody/CbcReports/ConstEntities/OtherEntityInfo_chi', 'Name': 'OtherEntityInfo_chi', 'Type': '', 'Depth': 4, 'Leaf': 'Y', 'Sequence': 22},
    # AdditionalInfo 區塊
    {'Path': '/CBC_OECD/CbcBody/AdditionalInfo', 'Name': 'AdditionalInfo', 'Type': 'TwCbcCorrectableAdditionalInfo_Type', 'Depth': 4, 'Leaf': 'N', 'Sequence': 6},
    {'Path': '/CBC_OECD/CbcBody/AdditionalInfo/OtherInfo', 'Name': 'OtherInfo', 'Type': 'TwCbcStringMin1Max4000WithLangEN_Type', 'Depth': 5, 'Leaf': 'Y', 'Sequence': 1},
    {'Path': '/CBC_OECD/CbcBody/AdditionalInfo/ResCountryCode', 'Name': 'ResCountryCode', 'Type': 'CountryCode_Type', 'Depth': 5, 'Leaf': 'Y', 'Sequence': 2},
    {'Path': '/CBC_OECD/CbcBody/AdditionalInfo/SummaryRef', 'Name': 'SummaryRef', 'Type': 'CbcSummaryListElementsType_EnumType', 'Depth': 5, 'Leaf': 'Y', 'Sequence': 3},
    {'Path': '/CBC_OECD/CbcBody/AdditionalInfoChi', 'Name': 'AdditionalInfoChi', 'Type': 'TwCbcCorrectableAdditionalInfoChi_Type', 'Depth': 4, 'Leaf': 'N', 'Sequence': 7},
    {'Path': '/CBC_OECD/CbcBody/AdditionalInfoChi/OtherInfo', 'Name': 'OtherInfo', 'Type': '', 'Depth': 5, 'Leaf': 'Y', 'Sequence': 1},

    #CbcCover 區塊
    {'Path': '/CBC_OECD/CbcCover', 'Name': 'CbcCover', 'Type': 'TwCbcCover_Type', 'Depth': 3, 'Leaf': 'N', 'Sequence': 8},
    {'Path': '/CBC_OECD/CbcCover/RegistrationInfo', 'Name': 'RegistrationInfo', 'Type': 'TwCbcRegistrationInfo_Type', 'Depth': 2, 'Leaf': 'N', 'Sequence': 1},
    {'Path': '/CBC_OECD/CbcCover/RegistrationInfo/BAN', 'Name': 'BAN', 'Type': 'TwCbcBAN_Type', 'Depth': 3, 'Leaf': 'Y', 'Sequence': 1},
    {'Path': '/CBC_OECD/CbcCover/RegistrationInfo/BANName', 'Name': 'BANName', 'Type': '', 'Depth': 3, 'Leaf': 'Y', 'Sequence': 2},
    {'Path': '/CBC_OECD/CbcCover/RegistrationInfo/HSNCD', 'Name': 'HSNCD', 'Type': 'TwCbcCityCodeType_EnumType', 'Depth': 5, 'Leaf': 'Y', 'Sequence': 3},
    {'Path': '/CBC_OECD/CbcCover/RegistrationInfo/VATLOSN', 'Name': 'VATLOSN', 'Type': 'TwCbcVATLOSN_Type', 'Depth': 5, 'Leaf': 'Y', 'Sequence': 4},
    {'Path': '/CBC_OECD/CbcCover/RegistrationInfo/BANAddress', 'Name': 'BANAddress', 'Type': '', 'Depth': 5, 'Leaf': 'Y', 'Sequence': 5},
    {'Path': '/CBC_OECD/CbcCover/RegistrationInfo/RespName', 'Name': 'RespName', 'Type': '', 'Depth': 5, 'Leaf': 'Y', 'Sequence': 6},
    {'Path': '/CBC_OECD/CbcCover/RegistrationInfo/RespAddress', 'Name': 'RespAddress', 'Type': '', 'Depth': 5, 'Leaf': 'Y', 'Sequence': 7},
    {'Path': '/CBC_OECD/CbcCover/RegistrationInfo/RespIDN', 'Name': 'RespIDN', 'Type': '', 'Depth': 5, 'Leaf': 'Y', 'Sequence': 8},
    {'Path': '/CBC_OECD/CbcCover/RegistrationInfo/RespIDNMk', 'Name': 'RespIDNMk', 'Type': '', 'Depth': 5, 'Leaf': 'Y', 'Sequence': 9},
    {'Path': '/CBC_OECD/CbcCover/RegistrationInfo/PhoneNumber', 'Name': 'PhoneNumber', 'Type': '', 'Depth': 5, 'Leaf': 'Y', 'Sequence': 10},
    {'Path': '/CBC_OECD/CbcCover/RegistrationInfo/SendMode', 'Name': 'SendMode', 'Type': '', 'Depth': 5, 'Leaf': 'Y', 'Sequence': 11},
    {'Path': '/CBC_OECD/CbcCover/AgentInfo', 'Name': 'AgentInfo', 'Type': 'TwCbcAgentInfo_Type', 'Depth': 4, 'Leaf': 'N', 'Sequence': 9},
    {'Path': '/CBC_OECD/CbcCover/AgentInfo/FilerName', 'Name': 'FilerName', 'Type': '', 'Depth': 5, 'Leaf': 'Y', 'Sequence': 1},
    {'Path': '/CBC_OECD/CbcCover/AgentInfo/FilerIDN', 'Name': 'FilerIDN', 'Type': '', 'Depth': 5, 'Leaf': 'Y', 'Sequence': 2},
    {'Path': '/CBC_OECD/CbcCover/AgentInfo/AgentIDNMk', 'Name': 'AgentIDNMk', 'Type': '', 'Depth': 5, 'Leaf': 'Y', 'Sequence': 3},
    {'Path': '/CBC_OECD/CbcCover/AgentInfo/FilerAddress', 'Name': 'FilerAddress', 'Type': '', 'Depth': 5, 'Leaf': 'Y', 'Sequence': 4},
    {'Path': '/CBC_OECD/CbcCover/AgentInfo/FilerTel', 'Name': 'FilerTel', 'Type': '', 'Depth': 5, 'Leaf': 'Y', 'Sequence': 5},
    {'Path': '/CBC_OECD/CbcCover/AgentInfo/LicenseType', 'Name': 'LicenseType', 'Type': '', 'Depth': 5, 'Leaf': 'Y', 'Sequence': 6},
    {'Path': '/CBC_OECD/CbcCover/AgentInfo/LicenseYear', 'Name': 'LicenseYear', 'Type': '', 'Depth': 5, 'Leaf': 'Y', 'Sequence': 7},
    {'Path': '/CBC_OECD/CbcCover/AgentInfo/LicenseNo', 'Name': 'LicenseNo', 'Type': '', 'Depth': 5, 'Leaf': 'Y', 'Sequence': 8},
    {'Path': '/CBC_OECD/CbcCover/AgentInfo/OrgnAgentHSNName', 'Name': 'OrgnAgentHSNName', 'Type': '', 'Depth': 5, 'Leaf': 'Y', 'Sequence': 9},
    {'Path': '/CBC_OECD/CbcCover/AgentInfo/OrgnAgentNo', 'Name': 'OrgnAgentNo', 'Type': '', 'Depth': 5, 'Leaf': 'Y', 'Sequence': 10},
    {'Path': '/CBC_OECD/CbcCover/AgentInfo/CommitDate', 'Name': 'CommitDate', 'Type': '', 'Depth': 5, 'Leaf': 'Y', 'Sequence': 11}


]


# ✅ Leaf 資料（只有 leaf 有值）
leaf_data = {
    "/CBC_OECD/MessageSpec/SendingEntityIN": "12345678",
    "/CBC_OECD/MessageSpec/currCode": "TWD",
    "/CBC_OECD/CbcBody/ReportingEntity/Entity/ResCountryCode": "TW",
    "/CBC_OECD/CbcBody/ReportingEntity/Entity/TIN": "12345678",
    "/CBC_OECD/CbcBody/ReportingEntity/Entity/IN": "",
    "/CBC_OECD/CbcBody/ReportingEntity/Entity/Name": "測試公司",
    "/CBC_OECD/CbcBody/ReportingEntity/Entity/Address/AddressFree": "Testing Company",
    "/CBC_OECD/CbcBody/ReportingEntity/NameMNEGroup": "Testing Group",
    "/CBC_OECD/CbcBody/ReportingEntity/NameMNEGroupCHi": "測試集團",
    "/CBC_OECD/CbcBody/ReportingEntity/ReportingRole": "CBC701",
    "/CBC_OECD/CbcBody/ReportingEntity/ReportingPeriod/StartDate": "2023-01-01",
    "/CBC_OECD/CbcBody/ReportingEntity/ReportingPeriod/EndDate": "2023-12-31",
    
    # 第一筆 CbcReports
    "/CBC_OECD/CbcBody/CbcReports[1]/ResCountryCode": "AT",
    "/CBC_OECD/CbcBody/CbcReports[1]/Summary/Revenues/Unrelated": "1000",
    "/CBC_OECD/CbcBody/CbcReports[1]/Summary/Revenues/Related": "1000",
    "/CBC_OECD/CbcBody/CbcReports[1]/Summary/Revenues/Total": "1000",
    "/CBC_OECD/CbcBody/CbcReports[1]/Summary/ProfitOrLoss": "1000",
    "/CBC_OECD/CbcBody/CbcReports[1]/Summary/TaxPaid": "1000",
    "/CBC_OECD/CbcBody/CbcReports[1]/Summary/TaxAccrued": "1000",
    "/CBC_OECD/CbcBody/CbcReports[1]/Summary/Capital": "1000",
    "/CBC_OECD/CbcBody/CbcReports[1]/Summary/Earnings": "1000",
    "/CBC_OECD/CbcBody/CbcReports[1]/Summary/NbEmployees": "1000",
    "/CBC_OECD/CbcBody/CbcReports[1]/Summary/Assets": "1000",

    "/CBC_OECD/CbcBody/CbcReports[1]/ConstEntities[1]/ConstEntity/ResCountryCode": "AT",
    "/CBC_OECD/CbcBody/CbcReports[1]/ConstEntities[1]/ConstEntity/TIN": "01234567",
    "/CBC_OECD/CbcBody/CbcReports[1]/ConstEntities[1]/ConstEntity/Name": "AT1",
    "/CBC_OECD/CbcBody/CbcReports[1]/ConstEntities[1]/ConstEntity/Address/AddressFree": "AT1 Address",
    "/CBC_OECD/CbcBody/CbcReports[1]/ConstEntities[1]/BizActivities[1]": "CBC513",
    "/CBC_OECD/CbcBody/CbcReports[1]/ConstEntities[1]/OtherEntityInfo_chi": "公司清算",

    "/CBC_OECD/CbcBody/CbcReports[1]/ConstEntities[2]/ConstEntity/ResCountryCode": "AT",
    "/CBC_OECD/CbcBody/CbcReports[1]/ConstEntities[2]/ConstEntity/TIN": "01234568",
    "/CBC_OECD/CbcBody/CbcReports[1]/ConstEntities[2]/ConstEntity/Name": "AT2",
    "/CBC_OECD/CbcBody/CbcReports[1]/ConstEntities[2]/ConstEntity/Address/AddressFree": "AT2 Address",
    "/CBC_OECD/CbcBody/CbcReports[1]/ConstEntities[2]/BizActivities[1]": "CBC501",
    "/CBC_OECD/CbcBody/CbcReports[1]/ConstEntities[2]/BizActivities[2]": "CBC502",
    "/CBC_OECD/CbcBody/CbcReports[1]/ConstEntities[2]/BizActivities[3]": "CBC503",

    # 第二筆 CbcReports
    "/CBC_OECD/CbcBody/CbcReports[2]/ResCountryCode": "TW",
    "/CBC_OECD/CbcBody/CbcReports[2]/Summary/Revenues/Unrelated": "2000",
    "/CBC_OECD/CbcBody/CbcReports[2]/Summary/Revenues/Related": "2000",
    "/CBC_OECD/CbcBody/CbcReports[2]/Summary/Revenues/Total": "2000",
    "/CBC_OECD/CbcBody/CbcReports[2]/Summary/ProfitOrLoss": "2000",
    "/CBC_OECD/CbcBody/CbcReports[2]/Summary/TaxPaid": "2000",
    "/CBC_OECD/CbcBody/CbcReports[2]/Summary/TaxAccrued": "2000",
    "/CBC_OECD/CbcBody/CbcReports[2]/Summary/Capital": "2000",
    "/CBC_OECD/CbcBody/CbcReports[2]/Summary/Earnings": "2000",
    "/CBC_OECD/CbcBody/CbcReports[2]/Summary/NbEmployees": "2000",
    "/CBC_OECD/CbcBody/CbcReports[2]/Summary/Assets": "2000",

    "/CBC_OECD/CbcBody/CbcReports[2]/ConstEntities[1]/ConstEntity/ResCountryCode": "TW",
    "/CBC_OECD/CbcBody/CbcReports[2]/ConstEntities[1]/ConstEntity/TIN": "11234567",
    "/CBC_OECD/CbcBody/CbcReports[2]/ConstEntities[1]/ConstEntity/Name": "TW1",
    "/CBC_OECD/CbcBody/CbcReports[2]/ConstEntities[1]/ConstEntity/Address/AddressFree": "TW1 Address",
    "/CBC_OECD/CbcBody/CbcReports[2]/ConstEntities[1]/BizActivities[1]": "CBC513",
    "/CBC_OECD/CbcBody/CbcReports[2]/ConstEntities[1]/OtherEntityInfo_chi": "公司清算",

    "/CBC_OECD/CbcBody/CbcReports[2]/ConstEntities[2]/ConstEntity/ResCountryCode": "TW",
    "/CBC_OECD/CbcBody/CbcReports[2]/ConstEntities[2]/ConstEntity/TIN": "11234568",
    "/CBC_OECD/CbcBody/CbcReports[2]/ConstEntities[2]/ConstEntity/Name": "TW2",
    "/CBC_OECD/CbcBody/CbcReports[2]/ConstEntities[2]/ConstEntity/Address/AddressFree": "TW2 Address",
    "/CBC_OECD/CbcBody/CbcReports[2]/ConstEntities[2]/BizActivities[1]": "CBC501",
    "/CBC_OECD/CbcBody/CbcReports[2]/ConstEntities[2]/BizActivities[2]": "CBC502",
    "/CBC_OECD/CbcBody/CbcReports[2]/ConstEntities[2]/BizActivities[3]": "CBC503",

    "/CBC_OECD/CbcBody/AdditionalInfo/OtherInfo": "The exchange rates adopted to translate the amount to group functional currency (TWD) are as follows: \n CNY(4.3995), EUR(33.6995), GBP(38.7528), JPY(0.2217), KRW(0.0239), MYR(6.838), USD(31.158), HKD(3.9804), PLN(7.4238), BRL(0.2002), DKK(4.5231), IDR(0.002), INR(0.3774), MXN(1.7577), SGD(23.2082), TRY(1.0929)",
    "/CBC_OECD/CbcBody/AdditionalInfoChi/OtherInfo": "將本幣金額換算為集團功能貨幣（新台幣）所採用的匯率如下: \n 人民幣(4.3995), 歐元(33.6995), 英鎊(38.7528), 日圓(0.2217), 韓圓(0.0239), 馬來西亞幣(6.838), 美金(31.158), 港幣(3.9804), 波蘭茲羅提(7.4238), 巴西雷亞爾(0.2002), 丹麥克朗(4.5231), 印尼盾(0.002), 印度盧比(0.3774), 墨西哥披索(1.7577), 新加坡幣(23.2082), 土耳其里拉(1.0929)",

    "/CBC_OECD/CbcCover/RegistrationInfo/BAN": "11223344",
    "/CBC_OECD/CbcCover/RegistrationInfo/BANName": "TEST公司",
    "/CBC_OECD/CbcCover/RegistrationInfo/HSNCD": "F",
    "/CBC_OECD/CbcCover/RegistrationInfo/VATLOSN": "123456789",
    "/CBC_OECD/CbcCover/RegistrationInfo/BANAddress": "台北市松仁路100號",
    "/CBC_OECD/CbcCover/RegistrationInfo/RespName": "王小明",
    "/CBC_OECD/CbcCover/RegistrationInfo/RespAddress": "台北市松仁路1號",
    "/CBC_OECD/CbcCover/RegistrationInfo/RespIDN": "A123456789",
    "/CBC_OECD/CbcCover/RegistrationInfo/RespIDNMk": "0",
    "/CBC_OECD/CbcCover/RegistrationInfo/PhoneNumber": "2882-5252",
    "/CBC_OECD/CbcCover/RegistrationInfo/SendMode": "1"
    
}

# 建立所有元素（支援 index）
cbcpath_element_map = {}

# 幫助比對不含 index 的 path
def strip_index(path):
    return re.sub(r"\[\d+\]", "", path)

# 從 leaf_data 收集所有實際使用過的 cbcpath（含 index）
actual_cbcpaths = set(leaf_data.keys())

# 所有需要建立的路徑（依照 leaf_data 裡的 index 展開）
all_needed_paths = set()

for path in actual_cbcpaths:
    parts = path.strip("/").split("/")
    for i in range(1, len(parts) + 1):
        partial_path = "/" + "/".join(parts[:i])
        all_needed_paths.add(partial_path)

# 依序建立元素
for path in sorted(all_needed_paths, key=lambda p: p.count("/")):
    base_path = strip_index(path)
    node = next((n for n in structure_sorted if n["Path"] == base_path), None)
    if not node:
        continue  # 無對應結構定義，略過

    tag = node["Name"]
    ns = QName(NSMAP["cbc"], tag)
    el = etree.Element(ns)

    # 放入文字（只有 leaf 有）
    if node["Leaf"] == "Y" and path in leaf_data:
        el.text = leaf_data[path]

    cbcpath_element_map[path] = el 

# 組裝 XML tree：照 structure_sorted 定義與實際 index 建立 parent-child 關係
children_map = defaultdict(list)

for path in sorted(cbcpath_element_map.keys(), key=lambda p: p.count("/")):
    if path == "/CBC_OECD":
        continue
    parent_path = "/" + "/".join(path.strip("/").split("/")[:-1])
    children_map[parent_path].append(path)

# 將子節點依據 structure_sorted 的順序加入 parent
for parent, children in children_map.items():
    # 依 Sequence 排序（需 strip index 比對）
    children.sort(key=lambda p: next(n["Sequence"] for n in structure_sorted if n["Path"] == strip_index(p)))
    for child_path in children:
        parent_el = cbcpath_element_map[parent]
        child_el = cbcpath_element_map[child_path]

        # 若是 leaf 且值為空，略過
        if child_path in leaf_data and (leaf_data[child_path] is None or str(leaf_data[child_path]).strip() == ""):
            continue
        parent_el.append(child_el)

        
# 建立 root，手動指定 namespace
root = etree.Element(QName(NSMAP["cbc"], "CBC_OECD"), nsmap=NSMAP)

# 設定 version 屬性與 schemaLocation
root.set("version", "1.0")
root.set(QName(NSMAP["xsi"], "schemaLocation"), "urn:oecd:ties:cbc:v2 ETAX-PIE-113.XSD")



# 掛上內容
cbc_root = cbcpath_element_map["/CBC_OECD"]
for child in cbc_root:
    root.append(child)

# 輸出 XML
xml_bytes = etree.tostring(
    root,
    pretty_print=True,
    xml_declaration=True,
    encoding="UTF-8"
)

output_path = r"C:\Users\waylin\mydjango\e_invoice\download\output_20250710.xml"

# 寫入檔案
with open("output_path", "wb") as f:
     f.write(xml_bytes)

# 印出結果
print(xml_bytes.decode("utf-8"))


<?xml version='1.0' encoding='UTF-8'?>
<cbc:CBC_OECD xmlns:cbc="urn:oecd:ties:cbc:v2" xmlns:iso="urn:oecd:ties:isocbctypes:v1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" version="1.0" xsi:schemaLocation="urn:oecd:ties:cbc:v2 ETAX-PIE-113.XSD">
  <cbc:MessageSpec>
    <cbc:SendingEntityIN>12345678</cbc:SendingEntityIN>
    <cbc:currCode>TWD</cbc:currCode>
  </cbc:MessageSpec>
  <cbc:CbcBody>
    <cbc:ReportingEntity>
      <cbc:Entity>
        <cbc:ResCountryCode>TW</cbc:ResCountryCode>
        <cbc:TIN>12345678</cbc:TIN>
        <cbc:Name>測試公司</cbc:Name>
        <cbc:Address>
          <cbc:AddressFree>Testing Company</cbc:AddressFree>
        </cbc:Address>
      </cbc:Entity>
      <cbc:NameMNEGroup>Testing Group</cbc:NameMNEGroup>
      <cbc:NameMNEGroupCHi>測試集團</cbc:NameMNEGroupCHi>
      <cbc:ReportingRole>CBC701</cbc:ReportingRole>
      <cbc:ReportingPeriod>
        <cbc:StartDate>2023-01-01</cbc:StartDate>
        <cbc:EndDate>2023-12-31</cbc:EndDate>
      </cbc:Rep